In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import numpy as np
import matplotlib.pyplot as plt
torch.set_default_dtype(torch.float64)



In [2]:
DEFAULT_WIDTH = 30
DEFAULT_DEPTH = 7

class PINN_f(nn.Module):
    def __init__(self, width=DEFAULT_WIDTH, depth=DEFAULT_DEPTH):
        super(PINN_f, self).__init__()
        self.width = width
        if depth < 2:
            raise ValueError(f"depth must be at least 2, got {depth}")
        self.depth = depth
        self.act = nn.Tanh()

        # layer definitions
        self.FC_dict = nn.ModuleDict()
        self.FC_dict[f"layer 1"] = nn.Linear(2, self.width)
        for i in range(2, depth):
            self.FC_dict[f"layer {i}"] = nn.Linear(self.width, self.width)
        self.FC_dict[f"layer {depth}"] = nn.Linear(self.width, 1)

    def forward(self, x):
        for key, layer in self.FC_dict.items():
            x = layer(x)
            if key != f"layer {self.depth}":
                x = self.act(x)
        return x


In [3]:
torch.manual_seed(0)
np.random.seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_phi = PINN_f().to(device)
model_psi = PINN_f().to(device)
model_w = PINN_f().to(device)
model_u1 = PINN_f().to(device)
model_u2 = PINN_f().to(device)

print(model_u1)
d_INIT = 3
l = torch.tensor([d_INIT,], dtype=torch.float64, device=device, requires_grad=True)
# define an optimizer
params = list(model_psi.parameters()) + list(model_w.parameters())+ list(model_phi.parameters()) + list(model_u1.parameters())+list(model_u2.parameters())


nn_optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=0)
#nn_scheduler = torch.optim.lr_scheduler.StepLR(nn_optimizer, step_size=1000, gamma=0.5)
l_optimizer = torch.optim.Adam([l], lr=0.01, weight_decay=0) #l learning rate can be even better
#l_scheduler = torch.optim.lr_scheduler.StepLR(l_optimizer, step_size=1000, gamma=0.5)

PINN_f(
  (act): Tanh()
  (FC_dict): ModuleDict(
    (layer 1): Linear(in_features=2, out_features=30, bias=True)
    (layer 2): Linear(in_features=30, out_features=30, bias=True)
    (layer 3): Linear(in_features=30, out_features=30, bias=True)
    (layer 4): Linear(in_features=30, out_features=30, bias=True)
    (layer 5): Linear(in_features=30, out_features=30, bias=True)
    (layer 6): Linear(in_features=30, out_features=30, bias=True)
    (layer 7): Linear(in_features=30, out_features=1, bias=True)
  )
)


In [4]:
# adam without smoothness loss, fixed l
batchsize = 5000 #interior
batchsize1 = 500 #boundary, smoothness? (close to the origin)
for ep in range(100000):
    nn_optimizer.zero_grad()

    #l_optimizer.zero_grad()

    ### equation loss
    # sample points uniformly in the transformed coordinates [-30,30]x[0,30]
    c = 30
    x1 = c*torch.rand(batchsize, 1, device=device)# (batch, 1)
    x1.requires_grad=True
    x2 = c*torch.rand(batchsize, 1, device=device) # (batch, 1)
    x2.requires_grad=True
    input1 = torch.cat([x1,x2], dim=-1) # (batch, 2)
    input2 = torch.cat([-x1,x2], dim=-1) # (batch, 2)
    phi = model_phi(input1)-model_phi(input2) # odd  parity
    psi = model_psi(input1)+model_psi(input2) #even
    w = model_w(input1)-model_w(input2) #odd
    u1 = model_u1(input1)-model_u1(input2) #odd
    u2 = model_u2(input1)+model_u2(input2) #even
    
    
    #compute the derivatives (only first orders)
    w_1 = grad(w.sum(), x1, create_graph=True)[0]
    w_2 = grad(w.sum(), x2, create_graph=True)[0]
    phi_1 = grad(phi.sum(), x1, create_graph=True)[0]
    phi_2 = grad(phi.sum(), x2, create_graph=True)[0]
    psi_1 = grad(psi.sum(), x1, create_graph=True)[0]
    psi_2 = grad(psi.sum(), x2, create_graph=True)[0]
    u1_1 = grad(u1.sum(), x1, create_graph=True)[0]
    u1_2 = grad(u1.sum(), x2, create_graph=True)[0]
    u2_1 = grad(u2.sum(), x1, create_graph=True)[0]
    u2_2 = grad(u2.sum(), x2, create_graph=True)[0]
    
    #compute the equation residue
    # note large/large might behave bad
    N_e = 6
    f_1 = w+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*w_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*w_2-phi
    f_2 = (2+u1_1/torch.cosh(x1))*phi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*phi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*phi_2+u2_1/torch.cosh(x1)*psi
    f_3 = (2+u2_2/torch.cosh(x2))*psi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*psi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*psi_2+u1_2/torch.cosh(x2)*phi
    f_4 = u1_1/torch.cosh(x1)+u2_2/torch.cosh(x2)
    f_5 = w-(u1_2/torch.cosh(x2)-u2_1/torch.cosh(x1))
    f_6 = phi_2/torch.cosh(x2)-psi_1/torch.cosh(x1)
    
    loss_i_1=torch.norm(f_1)**2/batchsize
    loss_i_2=torch.norm(f_2)**2/batchsize
    loss_i_3=torch.norm(f_3)**2/batchsize
    loss_i_4=torch.norm(f_4)**2/batchsize
    loss_i_5=torch.norm(f_5)**2/batchsize
    loss_i_6=torch.norm(f_6)**2/batchsize
    
    loss_i = (loss_i_1+loss_i_2+loss_i_3+loss_i_4+loss_i_5+loss_i_6)/N_e
    
    ###boundary loss
    # sample points
    y1 = c*torch.rand(batchsize1, 1, device=device)# (batch1, 1)
    y1.requires_grad=True
    y2 = c*torch.rand(batchsize1, 1, device=device) # (batch1, 1)
    y2.requires_grad=True
    y0 = torch.zeros(batchsize1,1, device=device,requires_grad=True)
    yd = c*torch.ones(batchsize1,1, device=device)
    yd.requires_grad=True
    
    #boundary 1
    input1_b1 = torch.cat([y1,y0], dim=-1) # (batch1, 2)
    input2_b1 = torch.cat([-y1,y0], dim=-1) # (batch1, 2)
    
    u2_b1 = model_u2(input1_b1)+model_u2(input2_b1) #even
    b_1 = u2_b1
    loss_b_1 = torch.norm(b_1)**2/batchsize1
    
    #boundary 2
    o1 = torch.zeros(1,1, device=device,requires_grad=True)
    o2 = torch.zeros(1,1, device=device,requires_grad=True)
    input_b2 = torch.cat([o1,o2], dim=-1)
    
    w_b2 = 2*model_w(input_b2)# odd derivative
    w_1_b2 = grad(w_b2.sum(), o1, create_graph=True)[0]
    b_2 = w_1_b2+1
    loss_b_2 = torch.norm(b_2)**2
    
    #boundary 3,5,7
    input1_b3 = torch.cat([yd,y2], dim=-1) # (batch1, 2)
    input2_b3 = torch.cat([-yd,y2], dim=-1) # (batch1, 2)
    
    u1_b3 = model_u1(input1_b3)-model_u1(input2_b3) #odd
    u2_b3 = model_u2(input1_b3)+model_u2(input2_b3) #even
    phi_b3 = model_phi(input1_b3)-model_phi(input2_b3)# odd
    psi_b3 = model_psi(input1_b3)+model_psi(input2_b3)# even
    u1_1_b3 = grad(u1_b3.sum(), yd, create_graph=True)[0]
    u1_2_b3 = grad(u1_b3.sum(), y2, create_graph=True)[0]
    u2_1_b3 = grad(u2_b3.sum(), yd, create_graph=True)[0]
    u2_2_b3 = grad(u2_b3.sum(), y2, create_graph=True)[0]
    b_31=u1_1_b3/torch.cosh(yd)
    b_33=u1_2_b3/torch.cosh(y2)
    b_34=u2_1_b3/torch.cosh(yd)
    b_32=u2_2_b3/torch.cosh(y2)
    b_5=phi_b3
    b_7=psi_b3
    loss_b_3 = (torch.norm(b_31)**2+torch.norm(b_32)**2+torch.norm(b_33)**2+torch.norm(b_34)**2+torch.norm(b_5)**2+torch.norm(b_7)**2)/batchsize1
    
    #boundary 4,6,8
    input1_b4 = torch.cat([torch.sinh(y1),torch.sinh(yd)], dim=-1) # (batch1, 2)
    input2_b4 = torch.cat([-torch.sinh(y1),torch.sinh(yd)], dim=-1) # (batch1, 2)
    
    u1_b4 = model_u1(input1_b4)-model_u1(input2_b4) #odd
    u2_b4 = model_u2(input1_b4)+model_u2(input2_b4) #even
    phi_b4 = model_phi(input1_b4)-model_phi(input2_b4)# odd
    psi_b4 = model_psi(input1_b4)+model_psi(input2_b4)# even
    u1_1_b4 = grad(u1_b4.sum(), y1, create_graph=True)[0]
    u1_2_b4 = grad(u1_b4.sum(), yd, create_graph=True)[0]
    u2_1_b4 = grad(u2_b4.sum(), y1, create_graph=True)[0]
    u2_2_b4 = grad(u2_b4.sum(), yd, create_graph=True)[0]
    b_41=u1_1_b4/torch.cosh(y1)
    b_43=u1_2_b4/torch.cosh(yd)
    b_44=u2_1_b4/torch.cosh(y1)
    b_42=u2_2_b4/torch.cosh(yd)
    b_6=phi_b4
    b_8=psi_b4
    loss_b_4 = (torch.norm(b_41)**2+torch.norm(b_42)**2+torch.norm(b_43)**2+torch.norm(b_44)**2+torch.norm(b_6)**2+torch.norm(b_8)**2)/batchsize1
    
    #collect the residue
    N_b=8
    loss_b = (loss_b_1+loss_b_2+loss_b_3+loss_b_4)/N_b

    #compute the equation residue
    f_1_s = w+((1+l)*torch.sinh(z1)+u1_s)/torch.cosh(z1)*w_1_s+((1+l)*torch.sinh(z2)+u2_s)/torch.cosh(z2)*w_2_s-phi_s
    f_2_s = (2+u1_1_s/torch.cosh(z1))*phi_s+((1+l)*torch.sinh(z1)+u1_s)/torch.cosh(z1)*phi_1_s+((1+l)*torch.sinh(z2)+u2_s)/torch.cosh(z2)*phi_2_s+u2_1_s/torch.cosh(z1)*psi_s
    f_3_s = (2+u2_2_s/torch.cosh(z2))*psi_s+((1+l)*torch.sinh(z1)+u1_s)/torch.cosh(z1)*psi_1_s+((1+l)*torch.sinh(z2)+u2_s)/torch.cosh(z2)*psi_2_s+u1_2_s/torch.cosh(z2)*phi_s
    f_4_s = u1_1_s/torch.cosh(z1)+u2_2_s/torch.cosh(z2)
    f_5_s = w-(u1_2_s/torch.cosh(z2)-u2_1_s/torch.cosh(z1))
    f_6_s = phi_2_s/torch.cosh(z2)-psi_1_s/torch.cosh(z1)
    
    #compute the residue derivatives
    F_1_1=grad(f_1_s.sum(), z1, create_graph=True)[0]
    F_1_2=grad(f_1_s.sum(), z2, create_graph=True)[0]
    F_2_1=grad(f_2_s.sum(), z1, create_graph=True)[0]
    F_2_2=grad(f_2_s.sum(), z2, create_graph=True)[0]
    F_3_1=grad(f_3_s.sum(), z1, create_graph=True)[0]
    F_3_2=grad(f_3_s.sum(), z2, create_graph=True)[0]
    F_4_1=grad(f_4_s.sum(), z1, create_graph=True)[0]
    F_4_2=grad(f_4_s.sum(), z2, create_graph=True)[0]
    F_5_1=grad(f_5_s.sum(), z1, create_graph=True)[0]
    F_5_2=grad(f_5_s.sum(), z2, create_graph=True)[0]
    F_6_1=grad(f_6_s.sum(), z1, create_graph=True)[0]
    F_6_2=grad(f_6_s.sum(), z2, create_graph=True)[0]
    

    loss_s_1=(torch.norm(F_1_1)**2+torch.norm(F_1_2)**2)/batchsize1
    loss_s_2=(torch.norm(F_2_1)**2+torch.norm(F_2_2)**2)/batchsize1
    loss_s_3=(torch.norm(F_3_1)**2+torch.norm(F_3_2)**2)/batchsize1
    loss_s_4=(torch.norm(F_4_1)**2+torch.norm(F_4_2)**2)/batchsize1
    loss_s_5=(torch.norm(F_5_1)**2+torch.norm(F_5_2)**2)/batchsize1
    loss_s_6=(torch.norm(F_6_1)**2+torch.norm(F_6_2)**2)/batchsize1
    
    loss_s = (loss_s_1+loss_s_2+loss_s_3+loss_s_4+loss_s_5+loss_s_6)/N_e
    
    
    #the total loss
    gamma=0.1
    loss =gamma*(loss_i+loss_s)+1*loss_b
    # update the model
    loss.backward()
    nn_optimizer.step()
    #nn_scheduler.step()

    #l_optimizer.step()
    #l_scheduler.step()

    # plot
    if ep % 10 == 0:
        print("Epoch is {},  function loss is {}, boundary loss is {},  overall loss is {} ".format(ep, loss_i.item(),loss_b.item(), loss.item()))
        #print(ep, 'loss' +{loss_i.item()}, loss_b.item(),loss1.item(),loss2.item(), loss.item())
        if ep % 100 == 0:
            print(l)
            # uniformly sample from [r,z] 
            nr=1000
            nz=500
            gridr = np.linspace(-20, 20,num=nr)
            gridz = np.linspace(0, 20, num=nz)
            rr, zz = np.meshgrid(gridr,gridz)
            input1 = np.stack([rr, zz], axis=2).reshape(nr*nz,2)
            input1 = torch.tensor([input1],dtype=torch.float64).to(device)
            input2 = np.stack([-rr, zz], axis=2).reshape(nr*nz,2)
            input2 = torch.tensor([input2],dtype=torch.float64).to(device)
            #print(input.shape)
            phi = model_phi(input1).reshape(nz,nr)-model_phi(input2).reshape(nz,nr) # odd  parity
            psi = model_psi(input1).reshape(nz,nr)+model_psi(input2).reshape(nz,nr) #even
            w = model_w(input1).reshape(nz,nr)-model_w(input2).reshape(nz,nr) #odd
            u1 = model_u1(input1).reshape(nz,nr)-model_u1(input2).reshape(nz,nr) #odd
            u2 = model_u2(input1).reshape(nz,nr)+model_u2(input2).reshape(nz,nr) #even


            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('u1')
            ax.plot_surface(rr, zz, u1.detach().cpu().numpy())
            plt.show()

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('u2')
            ax.plot_surface(rr, zz, u2.detach().cpu().numpy())
            plt.show()

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('w')
            ax.plot_surface(rr, zz, w.detach().cpu().numpy())
            plt.show()

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('phi')
            ax.plot_surface(rr, zz, phi.detach().cpu().numpy())
            plt.show()

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('psi')
            ax.plot_surface(rr, zz, psi.detach().cpu().numpy())
            plt.show()


NameError: name 'z1' is not defined

In [1]:
# adam input as function of y
batchsize = 4000 #interior
batchsize1 = 500 #boundary, smoothness? (close to the origin)
for ep in range(1000):
    nn_optimizer.zero_grad()

    #l_optimizer.zero_grad()

    ### equation loss
    # sample points uniformly in the transformed coordinates [-30,30]x[0,30]
    c = 30
    d = 2
    x11 = torch.sinh(c*torch.rand(batchsize, 1, device='cuda'))# (batch, 1)
    x12 = torch.sinh(d*torch.rand(batchsize, 1, device='cuda'))
    x1=torch.cat([x11,x12], dim=0)
    x1.requires_grad=True
    x21 = torch.sinh(c*torch.rand(batchsize, 1, device='cuda'))# (batch, 1)
    x22 = torch.sinh(d*torch.rand(batchsize, 1, device='cuda'))
    x2=torch.cat([x21,x22], dim=0) # (2batch, 1)
    x2.requires_grad=True
    input1 = torch.cat([x1,x2], dim=-1) # (2batch, 2)
    input2 = torch.cat([-x1,x2], dim=-1) # (2batch, 2)
    phi = model_phi(input1)-model_phi(input2) # odd  parity
    psi = model_psi(input1)+model_psi(input2) #eve
    w = model_w(input1)-model_w(input2) #odd
    u1 = model_u1(input1)-model_u1(input2) #odd
    u2 = model_u2(input1)+model_u2(input2) #even
    
    
    #compute the derivatives (only first orders)
    w_1 = grad(w.sum(), x1, create_graph=True)[0]
    w_2 = grad(w.sum(), x2, create_graph=True)[0]
    phi_1 = grad(phi.sum(), x1, create_graph=True)[0]
    phi_2 = grad(phi.sum(), x2, create_graph=True)[0]
    psi_1 = grad(psi.sum(), x1, create_graph=True)[0]
    psi_2 = grad(psi.sum(), x2, create_graph=True)[0]
    u1_1 = grad(u1.sum(), x1, create_graph=True)[0]
    u1_2 = grad(u1.sum(), x2, create_graph=True)[0]
    u2_1 = grad(u2.sum(), x1, create_graph=True)[0]
    u2_2 = grad(u2.sum(), x2, create_graph=True)[0]
    
    #compute the equation residue
    N_e = 6
    f_1 = w+((1+l)*(x1)+u1)*w_1+((1+l)*(x2)+u2)*w_2-phi
    f_2 = (2+u1_1)*phi+((1+l)*(x1)+u1)*phi_1+((1+l)*(x2)+u2)*phi_2+u2_1*psi
    f_3 = (2+u2_2)*psi+((1+l)*(x1)+u1)*psi_1+((1+l)*(x2)+u2)*psi_2+u1_2*phi
    f_4 = u1_1+u2_2
    f_5 = w-(u1_2-u2_1)
    f_6 = phi_2-psi_1
    
    loss_i_1=torch.norm(f_1)**2/batchsize
    loss_i_2=torch.norm(f_2)**2/batchsize
    loss_i_3=torch.norm(f_3)**2/batchsize
    loss_i_4=torch.norm(f_4)**2/batchsize
    loss_i_5=torch.norm(f_5)**2/batchsize
    loss_i_6=torch.norm(f_6)**2/batchsize
    
    loss_i = (loss_i_1+loss_i_2+loss_i_3+loss_i_4+loss_i_5+loss_i_6)/N_e
    
    ###boundary loss
    # sample points
    y1 =torch.sinh(c*torch.rand(batchsize1, 1, device='cuda'))# (batch1, 1)
    y1.requires_grad=True
    y2 = torch.sinh(c*torch.rand(batchsize1, 1, device='cuda')) # (batch1, 1)
    y2.requires_grad=True
    y0 = torch.zeros(batchsize1,1, device='cuda',requires_grad=True)
    yd = torch.sinh(c*torch.ones(batchsize1,1, device='cuda'))
    yd.requires_grad=True
    
    #boundary 1
    input1_b1 = torch.cat([(y1),(y0)], dim=-1) # (batch1, 2)
    input2_b1 = torch.cat([-(y1),(y0)], dim=-1) # (batch1, 2)
    
    u2_b1 = model_u2(input1_b1)+model_u2(input2_b1) #even
    b_1 = u2_b1
    loss_b_1 = torch.norm(b_1)**2/batchsize1
    
    #boundary 2
    o1 = torch.zeros(1,1, device='cuda',requires_grad=True)
    o2 = torch.zeros(1,1, device='cuda',requires_grad=True)
    input_b2 = torch.cat([o1,o2], dim=-1)
    
    w_b2 = 2*model_w(input_b2)# odd derivative
    w_1_b2 = grad(w_b2.sum(), o1, create_graph=True)[0]
    b_2 = w_1_b2+1
    loss_b_2 = torch.norm(b_2)**2
    
    #boundary 3,5,7
    input1_b3 = torch.cat([(yd),(y2)], dim=-1) # (batch1, 2)
    input2_b3 = torch.cat([-(yd),y2], dim=-1) # (batch1, 2)
    
    u1_b3 = model_u1(input1_b3)-model_u1(input2_b3) #odd
    u2_b3 = model_u2(input1_b3)+model_u2(input2_b3) #even
    phi_b3 = model_phi(input1_b3)-model_phi(input2_b3)# odd
    psi_b3 = model_psi(input1_b3)+model_psi(input2_b3)# even
    u1_1_b3 = grad(u1_b3.sum(), yd, create_graph=True)[0]
    u1_2_b3 = grad(u1_b3.sum(), y2, create_graph=True)[0]
    u2_1_b3 = grad(u2_b3.sum(), yd, create_graph=True)[0]
    u2_2_b3 = grad(u2_b3.sum(), y2, create_graph=True)[0]
    b_31=u1_1_b3
    b_33=u1_2_b3
    b_34=u2_1_b3
    b_32=u2_2_b3
    b_5=phi_b3
    b_7=psi_b3
    loss_b_3 = (torch.norm(b_31)**2+torch.norm(b_32)**2+torch.norm(b_33)**2+torch.norm(b_34)**2+torch.norm(b_5)**2+torch.norm(b_7)**2)/batchsize1
    
    #boundary 4,6,8
    input1_b4 = torch.cat([(y1),(yd)], dim=-1) # (batch1, 2)
    input2_b4 = torch.cat([-(y1),(yd)], dim=-1) # (batch1, 2)
    
    u1_b4 = model_u1(input1_b4)-model_u1(input2_b4) #odd
    u2_b4 = model_u2(input1_b4)+model_u2(input2_b4) #even
    phi_b4 = model_phi(input1_b4)-model_phi(input2_b4)# odd
    psi_b4 = model_psi(input1_b4)+model_psi(input2_b4)# even
    u1_1_b4 = grad(u1_b4.sum(), y1, create_graph=True)[0]
    u1_2_b4 = grad(u1_b4.sum(), yd, create_graph=True)[0]
    u2_1_b4 = grad(u2_b4.sum(), y1, create_graph=True)[0]
    u2_2_b4 = grad(u2_b4.sum(), yd, create_graph=True)[0]
    b_41=u1_1_b4
    b_43=u1_2_b4
    b_44=u2_1_b4
    b_42=u2_2_b4
    b_6=phi_b4
    b_8=psi_b4
    loss_b_4 = (torch.norm(b_41)**2+torch.norm(b_42)**2+torch.norm(b_43)**2+torch.norm(b_44)**2+torch.norm(b_6)**2+torch.norm(b_8)**2)/batchsize1
    
    #collect the residue
    N_b=8
    loss_b = (loss_b_1+loss_b_2+loss_b_3+loss_b_4)/N_b
    
        
    
    #the total loss
    gamma=0.1
    loss =gamma*(loss_i)+1*loss_b

    # update the model
    loss.backward()
    nn_optimizer.step()
    #nn_scheduler.step()

    #l_optimizer.step()
    #l_scheduler.step()

    # plot
    if ep % 1000 == 0:
        print("Epoch is {},  function loss is {}, boundary loss is {},  overall loss is {} ".format(ep, loss_i.item(),loss_b.item(), loss.item()))
        #print(ep, 'loss' +{loss_i.item()}, loss_b.item(),loss1.item(),loss2.item(), loss.item())
        print(l)
        # uniformly sample from [r,z] 
        nr=1000
        nz=500
        gridr = np.linspace(-20, 20,num=nr)
        gridz = np.linspace(0, 20, num=nz)
        rr, zz = np.meshgrid(gridr,gridz)
        input1 = np.stack([rr, zz], axis=2).reshape(nr*nz,2)
        input1 = torch.tensor([input1],dtype=torch.float64).cuda()
        input2 = np.stack([-rr, zz], axis=2).reshape(nr*nz,2)
        input2 = torch.tensor([input2],dtype=torch.float64).cuda()
        #print(input.shape)
        phi = model_phi(input1).reshape(nz,nr)-model_phi(input2).reshape(nz,nr) # odd  parity
        psi = model_psi(input1).reshape(nz,nr)+model_psi(input2).reshape(nz,nr) #even
        w = model_w(input1).reshape(nz,nr)-model_w(input2).reshape(nz,nr) #odd
        u1 = model_u1(input1).reshape(nz,nr)-model_u1(input2).reshape(nz,nr) #odd
        u2 = model_u2(input1).reshape(nz,nr)+model_u2(input2).reshape(nz,nr) #even

        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.set_title('u1')
        ax.plot_surface(rr, zz, u1.detach().cpu().numpy())
        plt.show()
        
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.set_title('u2')
        ax.plot_surface(rr, zz, u2.detach().cpu().numpy())
        plt.show()
        
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.set_title('w')
        ax.plot_surface(rr, zz, w.detach().cpu().numpy())
        plt.show()
        
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.set_title('phi')
        ax.plot_surface(rr, zz, phi.detach().cpu().numpy())
        plt.show()

        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.set_title('psi')
        ax.plot_surface(rr, zz, psi.detach().cpu().numpy())
        plt.show()


NameError: name 'nn_optimizer' is not defined

In [ ]:
# adam with smoothness loss, fixed l
batchsize = 4000 #interior
batchsize1 = 500 #boundary, smoothness? (close to the origin)
for ep in range(100000):
    nn_optimizer.zero_grad()

    #l_optimizer.zero_grad()

    ### equation loss
    # sample points uniformly in the transformed coordinates [-30,30]x[0,30]
    c = 30
    d = 2
    x11 = (c*torch.rand(batchsize, 1, device=device))# (batch, 1)
    x12 = (d*torch.rand(batchsize, 1, device=device))
    x1=torch.cat([x11,x12], dim=0)
    x1.requires_grad=True
    x21 = (c*torch.rand(batchsize, 1, device=device))# (batch, 1)
    x22 = (d*torch.rand(batchsize, 1, device=device))
    x2=torch.cat([x21,x22], dim=0) # (2batch, 1)
    x2.requires_grad=True
    input1 = torch.cat([x1,x2], dim=-1) # (batch, 2)
    input2 = torch.cat([-x1,x2], dim=-1) # (batch, 2)
    phi = model_phi(input1)-model_phi(input2) # odd  parity
    psi = model_psi(input1)+model_psi(input2) #even
    w = model_w(input1)-model_w(input2) #odd
    u1 = model_u1(input1)-model_u1(input2) #odd
    u2 = model_u2(input1)+model_u2(input2) #even
    
    
    #compute the derivatives (only first orders)
    w_1 = grad(w.sum(), x1, create_graph=True)[0]
    w_2 = grad(w.sum(), x2, create_graph=True)[0]
    phi_1 = grad(phi.sum(), x1, create_graph=True)[0]
    phi_2 = grad(phi.sum(), x2, create_graph=True)[0]
    psi_1 = grad(psi.sum(), x1, create_graph=True)[0]
    psi_2 = grad(psi.sum(), x2, create_graph=True)[0]
    u1_1 = grad(u1.sum(), x1, create_graph=True)[0]
    u1_2 = grad(u1.sum(), x2, create_graph=True)[0]
    u2_1 = grad(u2.sum(), x1, create_graph=True)[0]
    u2_2 = grad(u2.sum(), x2, create_graph=True)[0]
    
    #compute the equation residue
    N_e = 6
    f_1 = w+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*w_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*w_2-phi
    f_2 = (2+u1_1/torch.cosh(x1))*phi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*phi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*phi_2+u2_1/torch.cosh(x1)*psi
    f_3 = (2+u2_2/torch.cosh(x2))*psi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*psi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*psi_2+u1_2/torch.cosh(x2)*phi
    f_4 = u1_1/torch.cosh(x1)+u2_2/torch.cosh(x2)
    f_5 = w-(u1_2/torch.cosh(x2)-u2_1/torch.cosh(x1))
    f_6 = phi_2/torch.cosh(x2)-psi_1/torch.cosh(x1)
    
    loss_i_1=torch.norm(f_1)**2/batchsize
    loss_i_2=torch.norm(f_2)**2/batchsize
    loss_i_3=torch.norm(f_3)**2/batchsize
    loss_i_4=torch.norm(f_4)**2/batchsize
    loss_i_5=torch.norm(f_5)**2/batchsize
    loss_i_6=torch.norm(f_6)**2/batchsize
    
    loss_i = (loss_i_1+loss_i_2+loss_i_3+loss_i_4+loss_i_5+loss_i_6)/N_e
    
    ###boundary loss
    # sample points
    y1 = 2*c*(torch.rand(batchsize1, 1, device=device)-1/2)# (batch1, 1)
    y1.requires_grad=True
    y2 = c*torch.rand(batchsize1, 1, device=device) # (batch1, 1)
    y2.requires_grad=True
    y0 = torch.zeros(batchsize1,1, device=device,requires_grad=True)
    yd = c*torch.ones(batchsize1,1, device=device)
    yd.requires_grad=True
    
    #boundary 1
    input1_b1 = torch.cat([y1,y0], dim=-1) # (batch1, 2)
    input2_b1 = torch.cat([-y1,y0], dim=-1) # (batch1, 2)
    
    u2_b1 = model_u2(input1_b1)+model_u2(input2_b1) #even
    b_1 = u2_b1
    loss_b_1 = torch.norm(b_1)**2/batchsize1
    
    #boundary 2
    o1 = torch.zeros(1,1, device=device,requires_grad=True)
    o2 = torch.zeros(1,1, device=device,requires_grad=True)
    input_b2 = torch.cat([o1,o2], dim=-1)
    w_b2 = 2*model_w(input_b2)# odd derivative
    w_1_b2 = grad(w_b2.sum(), o1, create_graph=True)[0]
    b_2 = w_1_b2+1
    loss_b_2 = torch.norm(b_2)**2
    #boundary 3,5,7
    input1_b3 = torch.cat([yd,y2], dim=-1) # (batch1, 2)
    input2_b3 = torch.cat([-yd,y2], dim=-1) # (batch1, 2)
    
    u1_b3 = model_u1(input1_b3)-model_u1(input2_b3) #odd
    u2_b3 = model_u2(input1_b3)+model_u2(input2_b3) #even
    phi_b3 = model_phi(input1_b3)-model_phi(input2_b3)# odd
    psi_b3 = model_psi(input1_b3)+model_psi(input2_b3)# even
    u1_1_b3 = grad(u1_b3.sum(), yd, create_graph=True)[0]
    u1_2_b3 = grad(u1_b3.sum(), y2, create_graph=True)[0]
    u2_1_b3 = grad(u2_b3.sum(), yd, create_graph=True)[0]
    u2_2_b3 = grad(u2_b3.sum(), y2, create_graph=True)[0]
    b_31=u1_1_b3/torch.cosh(yd)
    b_33=u1_2_b3/torch.cosh(y2)
    b_34=u2_1_b3/torch.cosh(yd)
    b_32=u2_2_b3/torch.cosh(y2)
    b_5=phi_b3
    b_7=psi_b3
    loss_b_3 = (torch.norm(b_31)**2+torch.norm(b_32)**2+torch.norm(b_33)**2+torch.norm(b_34)**2+torch.norm(b_5)**2+torch.norm(b_7)**2)/batchsize1
    
    #boundary 4,6,8
    input1_b4 = torch.cat([y1,yd], dim=-1) # (batch1, 2)
    input2_b4 = torch.cat([-y1,yd], dim=-1) # (batch1, 2)
    
    u1_b4 = model_u1(input1_b4)-model_u1(input2_b4) #odd
    u2_b4 = model_u2(input1_b4)+model_u2(input2_b4) #even
    phi_b4 = model_phi(input1_b4)-model_phi(input2_b4)# odd
    psi_b4 = model_psi(input1_b4)+model_psi(input2_b4)# even
    u1_1_b4 = grad(u1_b4.sum(), y1, create_graph=True)[0]
    u1_2_b4 = grad(u1_b4.sum(), yd, create_graph=True)[0]
    u2_1_b4 = grad(u2_b4.sum(), y1, create_graph=True)[0]
    u2_2_b4 = grad(u2_b4.sum(), yd, create_graph=True)[0]
    b_41=u1_1_b4/torch.cosh(y1)
    b_43=u1_2_b4/torch.cosh(yd)
    b_44=u2_1_b4/torch.cosh(y1)
    b_42=u2_2_b4/torch.cosh(yd)
    b_6=phi_b4
    b_8=psi_b4
    loss_b_4 = (torch.norm(b_41)**2+torch.norm(b_42)**2+torch.norm(b_43)**2+torch.norm(b_44)**2+torch.norm(b_6)**2+torch.norm(b_8)**2)/batchsize1
    
    #collect the residue
    N_b=8
    loss_b = (loss_b_1+loss_b_2+loss_b_3+loss_b_4)/N_b
    
    ### smoothness loss
    # sample points uniformly in the transformed coordinates [-1,1]x[0,1]
    x1 = 2*(torch.rand(batchsize1, 1, device=device)-1/2)# (batch, 1)
    x1.requires_grad=True
    x2 = torch.rand(batchsize1, 1, device=device) # (batch, 1)
    x2.requires_grad=True
    input1_s = torch.cat([x1,x2], dim=-1) # (batch, 2)
    input2_s = torch.cat([-x1,x2], dim=-1) # (batch, 2)
    phi = model_phi(input1_s)-model_phi(input2_s) # odd  parity
    psi = model_psi(input1_s)+model_psi(input2_s) #even
    w = model_w(input1_s)-model_w(input2_s) #odd
    u1 = model_u1(input1_s)-model_u1(input2_s) #odd
    u2 = model_u2(input1_s)+model_u2(input2_s) #even
    

    w_1 = grad(w.sum(), x1, create_graph=True)[0]
    w_2 = grad(w.sum(), x2, create_graph=True)[0]
    phi_1 = grad(phi.sum(), x1, create_graph=True)[0]
    phi_2 = grad(phi.sum(), x2, create_graph=True)[0]
    psi_1 = grad(psi.sum(), x1, create_graph=True)[0]
    psi_2 = grad(psi.sum(), x2, create_graph=True)[0]
    u1_1 = grad(u1.sum(), x1, create_graph=True)[0]
    u1_2 = grad(u1.sum(), x2, create_graph=True)[0]
    u2_1 = grad(u2.sum(), x1, create_graph=True)[0]
    u2_2 = grad(u2.sum(), x2, create_graph=True)[0]
    
    #compute the equation residue
    N_e = 6
    f_1 = w+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*w_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*w_2-phi
    f_2 = (2+u1_1/torch.cosh(x1))*phi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*phi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*phi_2+u2_1/torch.cosh(x1)*psi
    f_3 = (2+u2_2/torch.cosh(x2))*psi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*psi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*psi_2+u1_2/torch.cosh(x2)*phi
    f_4 = u1_1/torch.cosh(x1)+u2_2/torch.cosh(x2)
    f_5 = w-(u1_2/torch.cosh(x2)-u2_1/torch.cosh(x1))
    f_6 = phi_2/torch.cosh(x2)-psi_1/torch.cosh(x1)
    

    
    #compute the residue derivatives
    F_1_1=grad(f_1.sum(), x1, create_graph=True)[0]
    F_1_2=grad(f_1.sum(), x2, create_graph=True)[0]
    F_2_1=grad(f_2.sum(), x1, create_graph=True)[0]
    F_2_2=grad(f_2.sum(), x2, create_graph=True)[0]
    F_3_1=grad(f_3.sum(), x1, create_graph=True)[0]
    F_3_2=grad(f_3.sum(), x2, create_graph=True)[0]
    F_4_1=grad(f_4.sum(), x1, create_graph=True)[0]
    F_4_2=grad(f_4.sum(), x2, create_graph=True)[0]
    F_5_1=grad(f_5.sum(), x1, create_graph=True)[0]
    F_5_2=grad(f_5.sum(), x2, create_graph=True)[0]
    F_6_1=grad(f_6.sum(), x1, create_graph=True)[0]
    F_6_2=grad(f_6.sum(), x2, create_graph=True)[0]
    

    loss_s_1=(torch.norm(F_1_1)**2+torch.norm(F_1_2)**2)/batchsize1
    loss_s_2=(torch.norm(F_2_1)**2+torch.norm(F_2_2)**2)/batchsize1
    loss_s_3=(torch.norm(F_3_1)**2+torch.norm(F_3_2)**2)/batchsize1
    loss_s_4=(torch.norm(F_4_1)**2+torch.norm(F_4_2)**2)/batchsize1
    loss_s_5=(torch.norm(F_5_1)**2+torch.norm(F_5_2)**2)/batchsize1
    loss_s_6=(torch.norm(F_6_1)**2+torch.norm(F_6_2)**2)/batchsize1
    
    loss_s = (loss_s_1+loss_s_2+loss_s_3+loss_s_4+loss_s_5+loss_s_6)/N_e
    
    
    #the total loss
    gamma=0.1
    loss =gamma*(loss_i+loss_s)+1*loss_b

    # update the model
    loss.backward()
    nn_optimizer.step()
    #nn_scheduler.step()

    #l_optimizer.step()
    #l_scheduler.step()

    # plot
    if ep % 10 == 0:
        print("Epoch is {},  function loss is {}, boundary loss is {}, smoothness loss is {}, overall loss is {} ".format(ep, loss_i.item(),loss_b.item(),loss_s.item(), loss.item()))
        #print(ep, 'loss' +{loss_i.item()}, loss_b.item(),loss1.item(),loss2.item(), loss.item())
        if ep % 100 == 0:
            print(l)
            # uniformly sample from [r,z] 
            nr=1001
            nz=501
            gridr = np.linspace(-20, 20,num=nr)
            gridz = np.linspace(0, 20, num=nz)
            rr, zz = np.meshgrid(gridr,gridz)
            x1=torch.tensor([np.arcsinh(rr).reshape(nr*nz,1)],dtype=torch.float64,requires_grad=True).to(device)
            x2=torch.tensor([np.arcsinh(zz).reshape(nr*nz,1)],dtype=torch.float64).to(device)
            input1 = torch.cat([x1,x2], dim=-1)
            input2 = torch.cat([-x1,x2], dim=-1)
            #print(input.shape)
            phi = model_phi(input1).reshape(nz,nr)-model_phi(input2).reshape(nz,nr) # odd  parity
            psi = model_psi(input1).reshape(nz,nr)+model_psi(input2).reshape(nz,nr) #even
            w = model_w(input1)-model_w(input2) #odd
            w_1 = grad(w.sum(), x1, create_graph=True)[0]
            w = w.reshape(nz,nr) #odd
            w_1 = w_1.reshape(nz,nr)
            u1 = model_u1(input1).reshape(nz,nr)-model_u1(input2).reshape(nz,nr) #odd
            u2 = model_u2(input1).reshape(nz,nr)+model_u2(input2).reshape(nz,nr) #even
            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('u1')
            ax.plot_surface(rr, zz, u1.detach().cpu().numpy())
            plt.show()

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('u2')
            ax.plot_surface(rr, zz, u2.detach().cpu().numpy())
            plt.show()

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('w')
            ax.plot_surface(rr, zz, w.detach().cpu().numpy())
            plt.show()

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('phi')
            ax.plot_surface(rr, zz, phi.detach().cpu().numpy())
            plt.show()

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.set_title('psi')
            ax.plot_surface(rr, zz, psi.detach().cpu().numpy())
            plt.show()



In [ ]:
# adam with smoothness loss, optimize l
batchsize = 1000 #interior
batchsize1 = 500 #boundary, smoothness? (close to the origin)
for ep in range(100000):
    nn_optimizer.zero_grad()

    l_optimizer.zero_grad()

    ### equation loss
    # sample points uniformly in the transformed coordinates [-30,30]x[-30,30]
    c = 30
    x1 = 2*c*(torch.rand(batchsize, 1, device=device)-1/2)# (batch, 1)
    x1.requires_grad=True
    x2 = c*torch.rand(batchsize, 1, device=device) # (batch, 1)
    x2.requires_grad=True
    input1 = torch.cat([x1,x2], dim=-1) # (batch, 2)
    input2 = torch.cat([-x1,x2], dim=-1) # (batch, 2)
    phi = model_phi(input1)-model_phi(input2) # odd  parity
    psi = model_psi(input1)+model_psi(input2) #even
    w = model_w(input1)-model_w(input2) #odd
    u1 = model_u1(input1)-model_u1(input2) #odd
    u2 = model_u2(input1)+model_u2(input2) #even
    
    
    #compute the derivatives (only first orders)
    w_1 = grad(w.sum(), x1, create_graph=True)[0]
    w_2 = grad(w.sum(), x2, create_graph=True)[0]
    phi_1 = grad(phi.sum(), x1, create_graph=True)[0]
    phi_2 = grad(phi.sum(), x2, create_graph=True)[0]
    psi_1 = grad(psi.sum(), x1, create_graph=True)[0]
    psi_2 = grad(psi.sum(), x2, create_graph=True)[0]
    u1_1 = grad(u1.sum(), x1, create_graph=True)[0]
    u1_2 = grad(u1.sum(), x2, create_graph=True)[0]
    u2_1 = grad(u1.sum(), x1, create_graph=True)[0]
    u2_2 = grad(u1.sum(), x2, create_graph=True)[0]
    
    #compute the equation residue
    N_e = 6
    f_1 = w+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*w_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*w_2-phi
    f_2 = (2+u1_1/torch.cosh(x1))*phi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*phi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*phi_2+u2_1/torch.cosh(x1)*psi
    f_3 = (2+u2_2/torch.cosh(x2))*psi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*psi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*psi_2+u1_2/torch.cosh(x2)*phi
    f_4 = u1_1/torch.cosh(x1)+u2_2/torch.cosh(x2)
    f_5 = w-(u1_2/torch.cosh(x2)-u2_1/torch.cosh(x1))
    f_6 = phi_2/torch.cosh(x2)-psi_1/torch.cosh(x1)
    
    loss_i_1=torch.norm(f_1)**2/batchsize
    loss_i_2=torch.norm(f_2)**2/batchsize
    loss_i_3=torch.norm(f_3)**2/batchsize
    loss_i_4=torch.norm(f_4)**2/batchsize
    loss_i_5=torch.norm(f_5)**2/batchsize
    loss_i_6=torch.norm(f_6)**2/batchsize
    
    loss_i = (loss_i_1+loss_i_2+loss_i_3+loss_i_4+loss_i_5+loss_i_6)/N_e
    
    ###boundary loss
    # sample points
    y1 = 2*c*(torch.rand(batchsize1, 1, device=device)-1/2)# (batch1, 1)
    y1.requires_grad=True
    y2 = c*torch.rand(batchsize1, 1, device=device) # (batch1, 1)
    y2.requires_grad=True
    y0 = torch.zeros(batchsize1,1, device=device,requires_grad=True)
    yd = c*torch.ones(batchsize1,1, device=device)
    yd.requires_grad=True
    
    #boundary 1
    input1_b1 = torch.cat([y1,y0], dim=-1) # (batch1, 2)
    input2_b1 = torch.cat([-y1,y0], dim=-1) # (batch1, 2)
    
    u2_b1 = model_u2(input1_b1)+model_u2(input2_b1) #even
    b_1 = u2_b1
    loss_b_1 = torch.norm(b_1)**2/batchsize1
    
    #boundary 2
    o1 = torch.zeros(1,1, device=device,requires_grad=True)
    o2 = torch.zeros(1,1, device=device,requires_grad=True)
    input_b2 = torch.cat([o1,o2], dim=-1)
    
    w_b2 = 2*model_w(input_b2)# odd derivative
    w_1_b2 = grad(w_b2.sum(), o1, create_graph=True)[0]
    b_2 = w_1_b2+1
    loss_b_2 = torch.norm(b_2)**2
    
    #boundary 3,5,7
    input1_b3 = torch.cat([yd,y2], dim=-1) # (batch1, 2)
    input2_b3 = torch.cat([-yd,y2], dim=-1) # (batch1, 2)
    
    u1_b3 = model_u1(input1_b3)-model_u1(input2_b3) #odd
    u2_b3 = model_u2(input1_b3)+model_u2(input2_b3) #even
    phi_b3 = model_phi(input1_b3)-model_phi(input2_b3)# odd
    psi_b3 = model_psi(input1_b3)+model_psi(input2_b3)# even
    u1_1_b3 = grad(u1_b3.sum(), yd, create_graph=True)[0]
    u2_2_b3 = grad(u2_b3.sum(), y2, create_graph=True)[0]
    b_31=u1_1_b3/torch.cosh(yd)
    b_32=u2_2_b3/torch.cosh(y2)
    b_5=phi_b3
    b_7=psi_b3
    loss_b_3 = (torch.norm(b_31)**2+torch.norm(b_32)**2+torch.norm(b_5)**2+torch.norm(b_7)**2)/batchsize1
    
    #boundary 4,6,8
    input1_b4 = torch.cat([y1,yd], dim=-1) # (batch1, 2)
    input2_b4 = torch.cat([-y1,yd], dim=-1) # (batch1, 2)
    
    u1_b4 = model_u1(input1_b4)-model_u1(input2_b4) #odd
    u2_b4 = model_u2(input1_b4)+model_u2(input2_b4) #even
    phi_b4 = model_phi(input1_b4)-model_phi(input2_b4)# odd
    psi_b4 = model_psi(input1_b4)+model_psi(input2_b4)# even
    u1_1_b4 = grad(u1_b4.sum(), y1, create_graph=True)[0]
    u2_2_b4 = grad(u2_b4.sum(), yd, create_graph=True)[0]
    b_41=u1_1_b4/torch.cosh(y1)
    b_42=u2_2_b4/torch.cosh(yd)
    b_6=phi_b4
    b_8=psi_b4
    loss_b_4 = (torch.norm(b_41)**2+torch.norm(b_42)**2+torch.norm(b_6)**2+torch.norm(b_8)**2)/batchsize1
    
    #collect the residue
    N_b=8
    loss_b = (loss_b_1+loss_b_2+loss_b_3+loss_b_4)/N_b
    
    ### smoothness loss
    # sample points uniformly in the transformed coordinates [-1,1]x[0,1]
    z1 = 2*(torch.rand(batchsize, 1, device=device)-1/2)# (batch, 1)
    z1.requires_grad=True
    z2 = torch.rand(batchsize, 1, device=device) # (batch, 1)
    z2.requires_grad=True
    input1_s = torch.cat([z1,z2], dim=-1) # (batch, 2)
    input2_s = torch.cat([-z1,z2], dim=-1) # (batch, 2)
    phi_s = model_phi(input1_s)-model_phi(input2_s) # odd  parity
    psi_s = model_psi(input1_s)+model_psi(input2_s) #even
    w_s = model_w(input1_s)-model_w(input2_s) #odd
    u1_s = model_u1(input1_s)-model_u1(input2_s) #odd
    u2_s = model_u2(input1_s)+model_u2(input2_s) #even
    
    
    #compute the derivatives (only first orders)
    w_1_s = grad(w_s.sum(), z1, create_graph=True)[0]
    w_2_s = grad(w_s.sum(), z2, create_graph=True)[0]
    phi_1_s = grad(phi_s.sum(), z1, create_graph=True)[0]
    phi_2_s = grad(phi_s.sum(), z2, create_graph=True)[0]
    psi_1_s = grad(psi_s.sum(), z1, create_graph=True)[0]
    psi_2_s = grad(psi_s.sum(), z2, create_graph=True)[0]
    u1_1_s = grad(u1_s.sum(), z1, create_graph=True)[0]
    u1_2_s = grad(u1_s.sum(), z2, create_graph=True)[0]
    u2_1_s = grad(u1_s.sum(), z1, create_graph=True)[0]
    u2_2_s = grad(u1_s.sum(), z2, create_graph=True)[0]
    
    #compute the equation residue
    f_1_s = w_s+((1+l)*torch.sinh(z1)+u1_s)/torch.cosh(z1)*w_1_s+((1+l)*torch.sinh(z2)+u2_s)/torch.cosh(z2)*w_2_s-phi_s
    f_2_s = (2+u1_1_s/torch.cosh(z1))*phi_s+((1+l)*torch.sinh(z1)+u1_s)/torch.cosh(z1)*phi_1_s+((1+l)*torch.sinh(z2)+u2_s)/torch.cosh(z2)*phi_2_s+u2_1_s/torch.cosh(z1)*psi_s
    f_3_s = (2+u2_2_s/torch.cosh(z2))*psi_s+((1+l)*torch.sinh(z1)+u1_s)/torch.cosh(z1)*psi_1_s+((1+l)*torch.sinh(z2)+u2_s)/torch.cosh(z2)*psi_2_s+u1_2_s/torch.cosh(z2)*phi_s
    f_4_s = u1_1_s/torch.cosh(z1)+u2_2_s/torch.cosh(z2)
    f_5_s = w_s-(u1_2_s/torch.cosh(z2)-u2_1_s/torch.cosh(z1))
    f_6_s = phi_2_s/torch.cosh(z2)-psi_1_s/torch.cosh(z1)
    
    #compute the residue derivatives
    F_1_1=grad(f_1_s.sum(), z1, create_graph=True)[0]
    F_1_2=grad(f_1_s.sum(), z2, create_graph=True)[0]
    F_2_1=grad(f_2_s.sum(), z1, create_graph=True)[0]
    F_2_2=grad(f_2_s.sum(), z2, create_graph=True)[0]
    F_3_1=grad(f_3_s.sum(), z1, create_graph=True)[0]
    F_3_2=grad(f_3_s.sum(), z2, create_graph=True)[0]
    F_4_1=grad(f_4_s.sum(), z1, create_graph=True)[0]
    F_4_2=grad(f_4_s.sum(), z2, create_graph=True)[0]
    F_5_1=grad(f_5_s.sum(), z1, create_graph=True)[0]
    F_5_2=grad(f_5_s.sum(), z2, create_graph=True)[0]
    F_6_1=grad(f_6_s.sum(), z1, create_graph=True)[0]
    F_6_2=grad(f_6_s.sum(), z2, create_graph=True)[0]
    

    loss_s_1=(torch.norm(F_1_1)**2+torch.norm(F_1_2)**2)/batchsize1
    loss_s_2=(torch.norm(F_2_1)**2+torch.norm(F_2_2)**2)/batchsize1
    loss_s_3=(torch.norm(F_3_1)**2+torch.norm(F_3_2)**2)/batchsize1
    loss_s_4=(torch.norm(F_4_1)**2+torch.norm(F_4_2)**2)/batchsize1
    loss_s_5=(torch.norm(F_5_1)**2+torch.norm(F_5_2)**2)/batchsize1
    loss_s_6=(torch.norm(F_6_1)**2+torch.norm(F_6_2)**2)/batchsize1
    
    loss_s = (loss_s_1+loss_s_2+loss_s_3+loss_s_4+loss_s_5+loss_s_6)/N_e
    
    
    #the total loss
    gamma=0.1
    loss =gamma*(loss_i+loss_s)+1*loss_b

    # update the model
    loss.backward()
    nn_optimizer.step()
    #nn_scheduler.step()

    l_optimizer.step()
    #l_scheduler.step()

    # plot
    if ep % 10 == 0:
        print("Epoch is {},  function loss is {}, boundary loss is {}, smoothness loss is {}, overall loss is {} ".format(ep, loss_i.item(),loss_b.item(),loss_s.item(), loss.item()))
        #print(ep, 'loss' +{loss_i.item()}, loss_b.item(),loss1.item(),loss2.item(), loss.item())
        print(l)
        # uniformly sample from [r,z] 
        if ep % 100 == 0:
            nr=1000
            nz=500
            gridr = np.linspace(-20, 20,num=nr)
            gridz = np.linspace(0, 20, num=nz)
            rr, zz = np.meshgrid(gridr,gridz)
            input1 = np.stack([rr, zz], axis=2).reshape(nr*nz,2)
            input1 = torch.tensor([input1]).to(device)
            input2 = np.stack([-rr, zz], axis=2).reshape(nr*nz,2)
            input2 = torch.tensor([input2]).to(device)
            #print(input.shape)
            phi = model_phi(input1).reshape(nz,nr)-model_phi(input2).reshape(nz,nr) # odd  parity
            psi = model_psi(input1).reshape(nz,nr)+model_psi(input2).reshape(nz,nr) #even
            w = model_w(input1).reshape(nz,nr)-model_w(input2).reshape(nz,nr) #odd
            u1 = model_u1(input1).reshape(nz,nr)-model_u1(input2).reshape(nz,nr) #odd
            u2 = model_u2(input1).reshape(nz,nr)+model_u2(input2).reshape(nz,nr) #even
            plt.imshow(u1.detach().cpu().numpy())
            plt.show()
            plt.imshow(u2.detach().cpu().numpy())
            plt.show()
            #phi = model_phi(input1)+model_phi(input2)-model_phi(input3)-model_phi(input4) # odd and even parity
            #u = model_u(input1)+model_u(input2)-model_u(input3)-model_u(input4)
            #w = model_w(input1)+model_w(input2)-model_w(input3)-model_w(input4)
            #plt.plot(input1.detach().cpu().numpy(), phi.detach().cpu().numpy())
            #plt.show()
            #plt.plot(input1.detach().cpu().numpy(), u.detach().cpu().numpy())
            #plt.show()
            #plt.plot(input1.detach().cpu().numpy(), w.detach().cpu().numpy())
            #plt.show()


In [ ]:
# check limiting behavior